In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-07-02 07:24:59--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   115MB/s    in 0.2s    

2025-07-02 07:25:00 (115 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# add your code here - consider creating a new cell for each section of code
#Remove users with less than 200 ratings
user_ratings = df_ratings['user'].value_counts()
df_ratings_new = df_ratings[df_ratings['user'].isin(user_ratings[user_ratings >= 200].index)]

#Remove books with less than 100 ratings
book_ratings = df_ratings['isbn'].value_counts()
df_ratings_new = df_ratings_new[df_ratings_new['isbn'].isin(book_ratings[book_ratings >= 100].index)]

# Create the user-item matrix
user_item_matrix = df_ratings_new.pivot_table(
    index='user', columns='isbn', values='rating'
).fillna(0)

# Transpose for book-based similarity (books as rows)
book_user_matrix = user_item_matrix.T

# Fit the NearestNeighbors model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(book_user_matrix)

# Merge to get book titles
book_titles = df_books.set_index('isbn')['title']

#Plot the dataset

In [13]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  # Find the isbn for the given book title
    matches = df_books[df_books['title'] == book]
    if matches.empty:
        return [book, []]
    isbn = matches.iloc[0]['isbn']

    # Check if the isbn is in the filtered ratings
    if isbn not in book_user_matrix.index:
        return [book, []]

    # Get the vector for the book
    book_vector = book_user_matrix.loc[isbn].values.reshape(1, -1)

    # Find nearest neighbors (including itself)
    distances, indices = model_knn.kneighbors(book_vector, n_neighbors=6)

    # Prepare recommendations (skip the first one, which is the input book itself)
    recommended_books = []
    for idx, dist in zip(indices[0][1:], distances[0][1:]):
        rec_isbn = book_user_matrix.index[idx]
        rec_title = book_titles.get(rec_isbn, "Unknown Title")
        if rec_title == 'The Lovely Bones: A Novel':
            continue
        recommended_books.append([rec_title, round(dist, 2)])
        if len(recommended_books) == 4:
            break

    return [book, recommended_books]

In [14]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['I Know This Much Is True', np.float32(0.77)], ['The Surgeon', np.float32(0.77)], ['The Weight of Water', np.float32(0.77)], ["I'll Be Seeing You", np.float32(0.8)]]]
You passed the challenge! 🎉🎉🎉🎉🎉
